In [1]:
from pymongo import MongoClient
from pprint import pprint

import pandas as pd
import re
import nltk

import pickle
import json

In [2]:
from bson.objectid import ObjectId

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# stemming
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /Users/Jocelyn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Jocelyn/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
# This creates a client that uses the default port on localhost.
# If connecting to AWS, you need a connection string.
# Can do the same thing with MongoClient("mongodb://localhost:27017")
# client = MongoClient()
client = MongoClient("mongodb://localhost:27017")

In [6]:
# Makes it look similar to shell mongo
db = client.amazon_lb

In [7]:
db.list_collection_names()

['product_reviews',
 'skin_care_face_rev_agg',
 'product_metadata',
 'skin_care_face_reviews',
 'skin_care_face_prod_rev',
 'skin_care_face_products']

In [8]:
# cursor = db.skin_care_face_products.find()
# products = list(cursor)
# products[0]

In [9]:
# cursor = db.skin_care_face_prod_rev.find()
# products = list(cursor)
# products[0]

In [10]:
# import product list
with open('pickles/product_list.pickle', 'rb') as to_read:
    product_list = pickle.load(to_read)

with open('pickles/review_list.pickle', 'rb') as to_read:
    review_list = pickle.load(to_read)
    
with open('pickles/all_review_list.pickle', 'rb') as to_read:
    all_review_list = pickle.load(to_read)
# product_list.head(5)

In [11]:
product_list.head(5)

,id,asin,product,overall_rating,review_count,description,category2,category3
0,5dc881769f9b98109203bca6,B0000Y3NO6,"DERMAdoctor Calm, Cool & Corrected anti-rednes...",3.9,34,,Treatments & Masks,Not listed
1,5dc881769f9b98109203bca9,B00012C5RS,"DERMAdoctor Picture Porefect Pore Minimizer, 1...",3.2,27,,"Exfoliators, Polishes & Scrubs",Exfoliators
2,5dc881769f9b98109203bcb9,B0001EKTTC,"Glytone Rejuvenating Mask, 3 oz.",4.3,26,Promotes a supple texture and bright tone Glyt...,Treatments & Masks,Masks
3,5dc881769f9b98109203bcc4,B0001EL5Q8,"PCA SKIN Rejuvenating Serum, 1 fl. oz.",3.6,55,Formulated with grape fruit stem cell extract ...,Treatments & Masks,Serums
4,5dc881769f9b98109203bcc5,B0001EL5JA,PCA SKIN Protecting Hydrator Broad Spectrum S...,4.3,41,This non-oily daily hydrator and sunscreen pr...,Creams & Moisturizers,Face Moisturizers


In [12]:
product_list[product_list['category2'] == 'Not listed']

,id,asin,product,overall_rating,review_count,description,category2,category3
227,5dc881769f9b98109203c231,B000SAH7SA,"Men Science Androceuticals Acne Spot Repair, 0...",4.0,8,Acne cream for men eliminates acne pimples in ...,Not listed,Not listed
253,5dc881769f9b98109203c27e,B000ULR8KE,"Pevonia RS2 Concentrate Rose, 1 Fl Oz",3.8,29,"Delivers immediate cooling, strengthens capill...",Not listed,Not listed
308,5dc881769f9b98109203c362,B000Z61VVQ,Juice Beauty SPF 30 Tinted Mineral Moisturizer...,3.9,334,This 4-in-1 multitasking moisturizer blends zi...,Not listed,Not listed
316,5dc881769f9b98109203c377,B000ZJO1YW,"Pevonia Ligne Funamentale ""C"" Complexe , 1 Fl Oz",4.1,48,Reveal a radiant complexion and fight signs of...,Not listed,Not listed
322,5dc881769f9b98109203c38c,B0010P6D28,"Revision Skincare Nectifirm, 1.7 oz",4.2,1510,Nectifirm is the #1 age-defying neck product r...,Not listed,Not listed
514,5dc881779f9b98109203c67e,B001I83RHG,"Juice Beauty Blemish Be Gone, 0.27 fl. oz.",3.4,21,Treat blemish prone areas anytime and anywhere...,Not listed,Not listed
526,5dc881779f9b98109203c6ab,B001KWW4YM,Yu-Be Foaming Skin Polish: Gently exfoliates u...,4.2,148,Fine grains of rice bran combine with bamboo a...,Not listed,Not listed
583,5dc881779f9b98109203c796,B001ULAKGW,boscia Clear Complexion Treatment,4.1,7,"A highly effective, lightweight gel formulated...",Not listed,Not listed
988,5dc881779f9b98109203cea8,B005WOIWFY,Personal Microderm Anti-Aging Recovery Moistur...,2.9,12,"Add fresh, clean moisture to your complexion w...",Not listed,Not listed
1156,5dc881789f9b98109203d211,B009K71VB6,Jouer Matte Moisture Tint,4.2,140,This lightweight mattifying moisture tint even...,Not listed,Not listed


In [13]:
product_list.iloc[10]['description']

'Night Therapy targets signs of visible damage for renewal during the restorative hours of sleep, optimizes moisture retention and relieves the consequential signs of photo aging An advanced formula, used nightly, helps to continuously improve the appearance of past photo damage and delivers high levels of hydration to promote overnight renewal. Key Performance Ingredients Copper Peptide Complex [CpC]: Also known as Bis (Tripeptide-1) Copper Acetate, a delivery system that uses the bodys natural protective carrier to deliver proprietary copper peptides, an essential micronutrient for skin health, for maximum restorative benefit. Endosomes: A liposome encapsulated extract from the marine microbe, Micrococcus Lysate. Extremely UV resistant, the extract contains the enzyme UV-endonuclease which improves the appearance of sun damaged skin. Ceramide Complex: A highly effective blend of Ceramides provides defense for the skins moisture barrier. Antioxidant Blend: An extremely potent combinat

In [14]:
# Text processing
# Text preprocessing steps - remove numbers, captial letters and punctuation
import re
import string

remove_links = lambda x: re.sub('<[^<]+?>', '', x)
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

#custom stop words
more_stop_words = lambda x: re.sub('fluid|fl|oz|ml|mls|ounce|ounces','',x)
more_stop_words2 = lambda x: re.sub('use|using|product|products|work|works|working|buy|buys|buying|stuff|good|excellent|great|bad|terrible|like|love|really','',x)

#custom replacements
mwe = lambda x: ' '.join(mwe_tokenizer.tokenize(word_tokenize(x)))

# product_list['description'] = product_list.description.map(remove_links).map(alphanumeric).map(punc_lower).map(more_stop_words)
product_list['product'] = product_list['product'].map(remove_links).map(alphanumeric).map(punc_lower).map(more_stop_words)


In [15]:
# Stemmers

stemmer = LancasterStemmer()
# stemmer = PorterStemmer
# stemmer = SnowballStemmer('english')
# lemmatizer = WordNetLemmatizer()

product_list['description']=[' '.join([stemmer.stem(word) for word in text.split(' ')])
          for text in product_list['description']]

In [33]:
# sentence tokenize to find similarities and remove
from nltk.tokenize import sent_tokenize

product_list['description2'] = [sent_tokenize(x) for x in product_list['description']]


In [37]:
product_list.iloc[4]['description2']

[' thi non-oily dai hydr and sunscreen provid light moist with a mat finish.',
 'act ingredients: directions: for the best protection, apply sunscreen every day, year-round, in every typ of weather.',
 'apply sunscreen 15 minut pri to expos to allow the act ingredy to penet the skin.',
 'form with beauty skin in mind our produc ar sci develop and car form to improv the heal and appear of yo un skin.',
 'we understand the import of form with the best combin of innov ingredy to provid you with safe, high effect produc that del on their promises.',
 'al of our produc ar fre of pot irrit lik synthet dye and fragrances.',
 'pca skin doe not perform or condon anim testing.',
 'pca skin is a trust innov in the develop of high effect skinc products.',
 'our vis is to improv peopl liv by provid results-oriented skin car solv for the heal of yo un skin.']

In [34]:
product_list.head(5)

,id,asin,product,overall_rating,review_count,description,category2,category3,description2
0,5dc881769f9b98109203bca6,B0000Y3NO6,dermadoctor calm cool corrected anti rednes...,3.9,34,,Treatments & Masks,Not listed,[]
1,5dc881769f9b98109203bca9,B00012C5RS,dermadoctor picture porefect pore minimizer ...,3.2,27,,"Exfoliators, Polishes & Scrubs",Exfoliators,[]
2,5dc881769f9b98109203bcb9,B0001EKTTC,glytone rejuvenating mask,4.3,26,"promot a suppl text and bright ton glytone, tr...",Treatments & Masks,Masks,"[promot a suppl text and bright ton glytone, t..."
3,5dc881769f9b98109203bcc4,B0001EL5Q8,pca skin rejuvenating serum,3.6,55,form with grap fruit stem cel extract and epid...,Treatments & Masks,Serums,[form with grap fruit stem cel extract and epi...
4,5dc881769f9b98109203bcc5,B0001EL5JA,pca skin protecting hydrator broad spectrum s...,4.3,41,thi non-oily dai hydr and sunscreen provid li...,Creams & Moisturizers,Face Moisturizers,[ thi non-oily dai hydr and sunscreen provid l...


In [17]:
# from nltk.tokenize import word_tokenize
# from nltk.tag import pos_tag
# from nltk.chunk import ne_chunk
# my_text = "James Smith lives in the United States."
# tokens = pos_tag(word_tokenize(my_text))
# tokens

In [18]:
### Vectorize title and description
tfidf_t = TfidfVectorizer(stop_words='english', min_df = 5)

vec_t = tfidf_t.fit_transform(product_list['product'])
matrix_t = pd.DataFrame(vec_t.toarray(),columns=tfidf_t.get_feature_names())
# matrix_t.head(5)
matrix_t.shape

(2233, 513)

In [19]:
matrix_t.head(5)

,absolute,acid,acids,acne,action,active,actives,advance,advanced,age,...,vitamin,vt,wash,water,white,wipes,wrinkle,wrinkles,youth,zirh
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
### Vectorize title and description
tfidf_d = TfidfVectorizer(stop_words='english', min_df = 5)

vec_d = tfidf_d.fit_transform(product_list['description'])
matrix_d = pd.DataFrame(vec_d.toarray(),columns=tfidf_d.get_feature_names())
# matrix_d.head(5)
matrix_d.shape

(2233, 2936)

In [21]:
matrix_d.head(5)

,000,10,100,11,114,12,13,14,15,150,...,youth,youthful,youthfulness,youv,zea,zero,zinc,zint,zone,zones
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.093961,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
product_list['category2'].nunique()

7

In [23]:
# Cluster by title
from sklearn.cluster import KMeans

km = KMeans(n_clusters=6)
km_trial = km.fit_transform(matrix_t)

In [24]:
type(km_trial)

numpy.ndarray

In [25]:
km_trial.shape

(2233, 6)

In [26]:
km_trial

array([[0.99855671, 1.04559247, 0.97153687, 1.19422449, 1.05421583,
        1.05424496],
       [1.00133883, 1.05839276, 1.04466231, 1.21034583, 1.05359196,
        1.05905326],
       [1.00393503, 1.05294326, 1.02924326, 1.19254136, 1.06326393,
        0.91831214],
       ...,
       [0.99819029, 1.05784434, 1.03880553, 1.21034583, 1.06637192,
        1.05993568],
       [0.99641452, 1.04856967, 1.0451208 , 1.21034583, 1.0587894 ,
        1.05727174],
       [0.97417525, 1.03861322, 1.0250473 , 1.18796701, 1.03795931,
        1.04651876]])

In [27]:
km_cluster = pd.DataFrame(km.fit_predict(km_trial),columns=['cluster'])

In [28]:
km_cluster['cluster'].value_counts()

5    1242
0     537
4     201
3     165
1      87
2       1
Name: cluster, dtype: int64

In [29]:
products1 = pd.merge(product_list, km_cluster, left_index=True, right_index=True)

In [30]:
products1.groupby(['category2','cluster']).count()

id  asin  product  overall_rating  \
category2                      cluster                                       
Cleansers                      0        113   113      113             113   
                               1         19    19       19              19   
                               3         22    22       22              22   
                               4         72    72       72              72   
                               5        213   213      213             213   
Creams & Moisturizers          0        196   196      196             196   
                               1         34    34       34              34   
                               3         54    54       54              54   
                               4         52    52       52              52   
                               5        426   426      426             426   
Exfoliators, Polishes & Scrubs 0         20    20       20              20   
                               1          3     3        3               3   
                               3          2     2        2               2   
                               4          8     8        8               8   
                               5         54    54       54              54   
Not listed                     0         11    11       11              11   
                               1          1     1        1               1   
                               3          2     2        2               2   
                               4          3     3        3               3   
                               5         23    23       23              23   
Sets & Kits                    0         22    22       22              22   
                               1          1     1        1               1   
                               3          8     8        8               8   
                               4          7     7        7               7   
                               5         49    49       49              49   
Toners & Astringents           0         26    26       26              26   
                               1          7     7        7               7   
                               3          9     9        9               9   
                               4          7     7        7               7   
                               5         76    76       76              76   
Treatments & Masks             0        146   146      146             146   
                               1         22    22       22              22   
                               2          1     1        1               1   
                               3         68    68       68              68   
                               4         50    50       50              50   
                               5        398   398      398             398   

                                        review_count  description  category3  
category2                      cluster                                        
Cleansers                      0                 113          113        113  
                               1                  19           19         19  
                               3                  22           22         22  
                               4                  72           72         72  
                               5                 213          213        213  
Creams & Moisturizers          0                 196          196        196  
                               1                  34           34         34  
                               3                  54           54         54  
                               4                  52           52         52  
                               5                 426          426        426  
Exfoliators, Polishes & Scrubs 0                  20           20         20  
                               1                 

### K-nearest neighbors model

In [40]:
product_list['category2'].value_counts()

Creams & Moisturizers             764
Treatments & Masks                687
Cleansers                         441
Toners & Astringents              126
Sets & Kits                        88
Exfoliators, Polishes & Scrubs     87
Not listed                         40
Name: category2, dtype: int64

In [42]:
for prod in product_list:
    if ('cream' in prod['title'].lower()) or ('moistur' in lower(prod['title'])):
        prod['category2'] == 'Creams & Moisturizers'
    elif 'mask' in lower(prod['title']):
        prod['category2'] == 'Treatments & Masks'
    elif 'cleans' in lower(prod['title']):
        prod['category2'] == 'Cleansers'
    elif ('toner' in lower(prod['title'])) or ('astring' in lower(prod['title'])):
        prod['category2'] == 'Toners & Astringents'
    elif ('set' in lower(prod['title'])) or ('kit' in lower(prod['title'])) or ('pack' in lower(prod['title'])):
        prod['category2'] == 'Sets & Kits'
    elif ('exfol' in lower(prod['title'])) or ('polish' in lower(prod['title'])) or ('scrub' in lower(prod['title'])):
        prod['category2'] == 'Exfoliators, Polishes & Scrubs'
   

TypeError: string indices must be integers